**AI Farming Buddy in Lesotho**

(*motsamaisi oa temo ea AI*) 

IEEE Humanitarian Technologies (IEEE HT) and the International Telecommunication Union (ITU) are harnessing generative AI to strengthen outcomes for smallholder farmers in Lesotho through a scalable and localized chatbot solution, in collaboration with the Food and Agriculture Organization of the United Nations (FAO). 

Through the [GenAI for Good Challenge](https://ieeeht.org/get-involved/funding-opportunities/genai-for-good/ ), IEEE HT and ITU are seeking prototype an AI-powered solutions that can provide and scale timely information for agricultural advisors and smallholder farmers in Lesotho. Agriculture employs nearly one-third of Lesotho’s workforce. But farmers and agricultural advisors face limited services, scarce digital tools, and mounting climate pressures. Without new solutions, **food security and rural resilience will remain at risk**. (see Geography and Climate in [Lesotho](https://en.wikipedia.org/wiki/Lesotho))

**AI Farming Buddy** attempts to build a single pane of glass  with LLM Agent personalized recommendations for farmers. It has MCP tool to collect current local weather information, tools to extract farming related metrics, and LLM agents to orchestrate these tools to generate  recommendations.  


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Observability: Logging configuration 

In [ ]:
import logging
import os

# Clean up any previous logs
for log_file in ["logger.log", "web.log", "tunnel.log"]:
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"🧹 Cleaned up {log_file}")

# Configure logging with DEBUG log level.
logging.basicConfig(
    filename="logger.log",
    level=logging.DEBUG,
    format="%(filename)s:%(lineno)s %(levelname)s:%(message)s",
)

print("✅ Logging configured")

In [2]:
!pip install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12

In [4]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [5]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [196]:
import datetime

def get_current_date_and_time()->dict:
    """Tool: Returns the current date and time

    Args:

    Returns:
        Dictionary with status and current_date_and_time value 
        Success: {"status": "success", "current_date_and_time": "2025-11-28T03:00"}
        Error: {"status": "error", "error_message": ""}
    """
    utc_dt = datetime.datetime.utcnow()
    print(f"{utc_dt.year}-{utc_dt.month:02d}-{utc_dt.day:02d}T{utc_dt.hour:02d}:{utc_dt.minute:02d}")
    return {'status': 'success',
            'current_date_and_time': f"{datetime.datetime.utcnow()}"
           }     

print("✅ get_current_date_and_time function created")
print(f"💱 Test: {get_current_date_and_time()}")

✅ get_current_date_and_time function created
2025-12-01T04:38
💱 Test: {'status': 'success', 'current_date_and_time': '2025-12-01 04:38:41.955031'}


In [203]:
def hitl_collect_input(message: str)->dict:
    """Ask user's input

    Args:
        message: 
        
    Returns:
         Dictionary with status and  
        Success: {"status": "success", "response": "my name is Bob."}
        Error: {"status": "error", "error_message": ""}

    """
    response = input(message)
    return {"status":"success", "response": response}
    

In [206]:
prompt1="What is your name?"
ans = hitl_collect_input(prompt1)
ans

What is your name? my name is bob


{'status': 'success', 'response': 'my name is bob'}

# Real-Time Weather MCP Server

This service provides various weather data for given location. 


## Get Current Temperature 

Retrieves the current temperature of given location

In [ ]:
import requests
from pydantic import BaseModel, Field
import datetime

def get_current_temperature_tool(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates.
    
    Args:
        latitude: 
        longitude:
        
    Returns:
        Dictionary with status and rate information.
        Success: {"status": "success", "gdd_accumulated": 0.93}
        Error: {"status": "error", "error_message": "Field does not exist"}

    """
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    reply = {}
    if response.status_code != 200:
         reply = {
            "status": "error",
            "error_message": f"API Request failed with status code: {response.status_code}",
        }
    else:
        results = response.json()

        current_utc_time = datetime.datetime.utcnow()
        time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
        temperature_list = results['hourly']['temperature_2m']
    
        closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
        current_temperature = temperature_list[closest_time_index]
        reply = {"status": "success", "current_temperature": current_temperature}
        
    return reply

print("✅ get_current_temperature_tool function created")
print(f"💱 Test: {get_current_temperature_tool(28,28)}")

### Create mcp_server directory

Source code of two MCP Server will be stored in ./mcp_server directory 

In [182]:
!mkdir mcp_server
!ls -al

total 196
drwxr-xr-x 5 root root   4096 Dec  1 04:26 .
drwxr-xr-x 5 root root   4096 Nov 30 21:50 ..
-rw-r--r-- 1 root root  12288 Dec  1 03:42 ai-farmer-buddy.db
-rw-r--r-- 1 root root 161733 Dec  1 04:23 ai-farmer-buddy.db.wal
drwxr-xr-x 2 root root   4096 Dec  1 00:57 .gradio
drwxr-xr-x 2 root root   4096 Dec  1 04:26 mcp_server
drwxr-xr-x 2 root root   4096 Nov 30 21:50 .virtual_documents


## Write Real-Time Weather MCP Server code 

In [ ]:
%%writefile mcp_server/weather_server.py
import logging
import os
import json
#import httpx
from mcp.server.fastmcp import FastMCP
import requests
from pydantic import BaseModel, Field
import datetime

mcp = FastMCP("real-time-weather-mcp")

@mcp.tool()
async def get_current_temperature(latitude: float, longitude: float) -> dict:
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    reply = {}
    if response.status_code != 200:
         reply = {
            "status": "error",
            "error_message": f"API Request failed with status code: {response.status_code}",
        }
    else:
        results = response.json()

        current_utc_time = datetime.datetime.utcnow()
        time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
        temperature_list = results['hourly']['temperature_2m']
    
        closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
        current_temperature = temperature_list[closest_time_index]
        reply = {"status": "success", "current_temperature": current_temperature}
        
    return reply


async def shutdown_event():
    logging.info("Shutting down MCP Server")
    pass

if __name__ == "__main__":
    mcp_server_log_dir = "./mcp_server"
    # Clean up any previous logs
    for log_file in ["./mcp_server/logger.log", "./mcp_server/web.log", "./mcp_server/tunnel.log"]:
        if os.path.exists(log_file):
            os.remove(log_file)
            print(f"🧹 Cleaned up {log_file}")

    # Configure logging with DEBUG log level.
    logging.basicConfig(
        filename="./mcp_server/logger.log",
        level=logging.DEBUG,
        format="%(filename)s:%(lineno)s %(levelname)s:%(message)s",
    )
    ## 
    logging.info("Starting MCP Server")
    mcp.run(transport="stdio")

## Validate MCP Server code

 1. Validate file exist 
 1. Validate the content of the file
 1. Validate by listing tools on MCP server
 1. Validate by calling a tool of MCP server


In [42]:
!ls -al mcp_server/*

ls: cannot access 'mcp_server/*': No such file or directory


In [ ]:
!cat ./mcp_server/weather_server.py

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# Create server parameters for stdio connection
weather_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your weather_server.py file
    args=["./mcp_server/weather_server.py"],
)

In [ ]:
weather_server_params

In [ ]:
!ls -al ./mcp_server

## Test MCP Server by using StdioServer

In [ ]:
import asyncio
from typing import Optional
from contextlib import AsyncExitStack

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

exit_stack = AsyncExitStack()
stdio_transport = await exit_stack.enter_async_context(stdio_client(weather_server_params))
stdio, write = stdio_transport
session = await exit_stack.enter_async_context(ClientSession(stdio, write))
    
await session.initialize()
    
# List available tools
response = await session.list_tools()
tools = response.tools
print("\nConnected to server with tools:", [tool.name for tool in tools])

available_tools = [{ 
        "name": tool.name,
        "description": tool.description,
        "input_schema": tool.inputSchema
    } for tool in response.tools]

print(available_tools)
tool_name = 'get_current_temperature'
tool_args = { 'latitude': 28, 'longitude':28 }
result = await session.call_tool(tool_name, tool_args)

tool_results = {"call": tool_name, "result": result}
final_text = f"[Calling tool {tool_name} with args {tool_args}]"
print(final_text)
print(tool_results)

In [36]:
from google.adk.tools.mcp_tool import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

# MCP integration with Real-Time Weather Server
mcp_rtweather_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="python",  # Run MCP server via python 
            args=[ "./mcp_server/weather_server.py",],
            #tool_filter=["getTinyImage"],
        ),
        timeout=30,
    )
)

print("✅ MCP Tool created")

✅ MCP Tool created


In [8]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

# Real-Time Weather Agent (utilizing Real-Time Weather MCP Server)

In [37]:
from google.adk.agents import LlmAgent

# Weather agent with custom function tools
instruction_prompt="""
 Use 'get_current_temperature()' to find the current temperature at given location by 'latitude' and 'longitude'. 
"""

weather_agent = LlmAgent(
    name="weather_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction=instruction_prompt,
    tools=[mcp_rtweather_server, ],
)

In [ ]:
# Test the real-time weather agent
weather_runner = InMemoryRunner(agent=weather_agent)
_ = await weather_runner.run_debug(
    "What is the current temparature at 'latitude': 28, 'longitude':28?"
)

### Retrieves historical data of given location

Utilize geohash and DuckDB

Using geohash of field's location to collect all data we can get from weather API

Using DuckDB to accumulate historical data. Grain is hourly and rolled upto daily when needed. 

In [135]:
!pip install duckdb
!pip install python-geohash

  Preparing metadata (setup.py) ... done
  Created wheel for python-geohash: filename=python_geohash-0.8.5-cp311-cp311-linux_x86_64.whl size=41947 sha256=99a2f4271f431347536e0b0ec43e92d3c42a71cf4e6cd0c8219be83f3ec18099
  Stored in directory: /root/.cache/pip/wheels/02/7a/f4/c27d535af1a4ad8c1112e1bf299d9d47b57fe0fa2a464e4795
Successfully built python-geohash


In [202]:
import requests
from pydantic import BaseModel, Field
import datetime

def get_historical_weather_tool(latitude: float, longitude: float, past_days: int) -> dict:
    """Fetch histotical weather measurements for a given coordinates.

    Collects hourly measurements
    
    Args:
        latitude: latitude of location 
        longitude: longitude of location
        past_days: size of look back window in days 
        
    Returns:
        Dictionary with status and rate information.
        Success: {"status": "success", "results": }
        Error: {"status": "error", "error_message": "Field does not exist"}

    """
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m,relative_humidity_2m,wind_speed_10m',
        'past_days': past_days,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    reply = {}
    if response.status_code != 200:
         reply = {
            "status": "error",
            "error_message": f"API Request failed with status code: {response.status_code}",
        }
    else:
        results = response.json()
        #print(results)
        
        current_utc_time = datetime.datetime.utcnow()
        time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
        temperature_list = results['hourly']['temperature_2m']
    
        closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
        current_temperature = temperature_list[closest_time_index]
        reply = {"status": "success", "results": results}
        
    return reply

print("✅ get_historical_weather_tool function created")
#print(f"💱 Test: {get_historical_weather_tool(28,28)}")

✅ get_historical_weather_tool function created


## MCP Server (Field Metrics)


### Accumulated Growing Degree Days (GDD)
[GDD](https://en.wikipedia.org/wiki/Growing_degree-day) is calculated using the daily maximum and minimum temperatures relative to a specific Base Temperature ($T_{base}$), which is the minimum temperature required for the organism (plant, pest, or disease) to grow.

Accumulated GDD is the sum of daily GDD since the planting date. 

| Organism  | $T_{base}$  |
|-----------|-------------|
| Tomato    | 10°C (50°F) |
| Patato    |  7°C (45°F)  |

In [ ]:
def get_gdd_accumulated_tool(field_id: str) -> dict:
    """MCP Tool: Retrieves persistent GDD memory from the database.

    Args:
        field_id: 
        
    Returns:
        Dictionary with status and rate information.
        Success: {"status": "success", "gdd_accumulated": 0.93}
        Error: {"status": "error", "error_message": "Field does not exist"}
    """
    print(f"[MCP Memory] Retrieving GDD for {field_id}...")
    gdd_accumulated =  620.0 # Tomato is in the flowering stage
    gdd_response = {}
    if gdd_accumulated is not None:
        gdd_response = {"status": "success", "gdd_accumulated": gdd_accumulated}
    else:
        gdd_response = {
            "status": "error",
            "error_message": f"Field ({field_id}) does not exist!",
        }
    return gdd_response 

print("✅ GDD Accumulated function created")
print(f"💱 Test: {get_gdd_accumulated_tool('field_1')}")

### Blitecast Severity Value

Calculates the Daily Severity Value (SV) based on Blitecast rules.


In [192]:
from typing import Union, List, TypeVar
import numpy as np

def get_blitecast_sv24(relativehumidity_hours: List[float],avg_temp_hours: List[float])->int:
    """Calculates the Daily Severity Value (SV) based on Blitecast rules.
    
    Args:
        relativehumidity_hours: Hourly Relative Humity in the last 24 hours
        avg_temp_hours: Hourly Average Temperature (F) in the last 24 fours
    
    Returns:
        int: The assigned Severity Value (0, 1, 2, or 3).
    """
    if(len(relativehumidity_hours)<10):
        return 0
    isRHHigh = [False for i in range(len(relativehumidity_hours))]
    #print(isRHHigh)
    isRHHigh = [True if val>=90 else False for val in relativehumidity_hours ]
    #print(isRHHigh)
    count_high = isRHHigh.count(True)
    print(f"count_high = {count_high}")
    
    if 10 <= count_high <= 15:
        avg_temp = np.mean(avg_temp_hours)
        print(f"avg_temp = {avg_temp}")
        if 45 <= avg_temp <= 54:
            return 1
        elif 55 <= avg_temp <= 77:
            return 2
        else:
            return 0
    elif count_high >= 16:
        avg_temp = np.mean(avg_temp_hours)
        print(f"avg_temp = {avg_temp}")
        if 45 <= avg_temp <= 54:
            return 2
        elif 55 <= avg_temp <= 77:
            return 3
        else:
            return 0
    
    return 0


In [193]:
import duckdb
import geohash
from datetime import datetime, timedelta

def get_blitecast_sv(field_id: int)-> dict:
    """Calculates Blite Cast Severity Value of the given field 

        Args:
            field_id : Field id in farmers_fields table
            
        Returns:
            int [0,1,2,3]
            Dictionary with status and Blite Cast Severity Value.
            Success: {"status": "success", "number_of_samples":127, "BliteCast_SV": 0}
              Error: {"status": "error", "error_message": "Field does not exist"}
    """
    ## Find field 
    con = duckdb.connect('ai-farmer-buddy.db')
    query_fetch = f"""
        SELECT id
            , latitude 
            , longitude
        FROM farmers_fields 
        WHERE id='{field_id}'
    """
    reply = con.sql(query_fetch).df()
    if len(reply)<1: 
        return {"staus":"error", "error_message":f"Field ({field_id}) does not exist"}
    ##     
    geohash_val = geohash.encode(reply.iloc[0]['latitude'], reply.iloc[0]['longitude'], precision=7)
    #print(f"({reply.iloc[0]['latitude']}, {reply.iloc[0]['longitude']}) => {geohash_val}")

    ## Find data of field
    yesterday_utc_time = datetime.today() - timedelta(days=1)
    y_time = yesterday_utc_time
    str_dtime=f"{y_time.year}-{y_time.month:02d}-{y_time.day:02d}T{y_time.hour:02d}:00:00"
    query_fetch = f"""
        SELECT id
            , ts
            , temperature_2m
            , relative_humidity_2m
            , wind_speed_10m 
        FROM historical_measurements
        WHERE geohash='{geohash_val}'
            and ts >= '{str_dtime}'
    """
    reply = con.sql(query_fetch).df()
    if len(reply)<1:
        return {"staus":"error", "error_message": f"Field ({field_id}) does not have enough data!"}
    val = get_blitecast_sv24(reply['relative_humidity_2m'].tolist(),reply['temperature_2m'].tolist())    
    return { "status": "success", "number_of_samples":len(reply), "BliteCast_SV": val }

In [31]:
# get_accumlated_gdd() depends on this function to calc accumulated GDD
def get_gdd(t_min: List[float], t_max: List[float], t_base: float)-> float:
    """Calculate Growing Degree Days 

        Daily metric.
        
    Args:
        t_min (float): Daily minimum temperature (°C).
        t_max (float): Daily maximum temperature (°C).
        t_base (float): Base temperature for the organism (°C).

    Returns:
        float : Daily GDD
    """
    if t_base is None:
        t_base=7.0
    t_diff = [ (((mx - mn)/2)-t_base) for mx, mn in zip(t_max, t_min)]        
    return max(0.0, sum(t_diff))

In [32]:
import duckdb
import datetime
import geohash

t_base_lookup = {"tomato":7, "potato":10}

def get_accumlated_gdd(field_id: int)-> dict:
    """Calculates Accumulated GDD for the given field 

    Args:
        field_id : Field identifier in farmers_fields table
        
    Returns:
        Dictionary with status and Accumulated GDD Value.
        Success: {"status": "success", "number_of_samples":127, "acc_gdd": 0}
          Error: {"status": "error", "error_message": "Field does not exist"}

    """
    ## Find field 
    con = duckdb.connect('ai-farmer-buddy.db')
    query_fetch = f"""
        SELECT id
            , latitude 
            , longitude
            , plant_ts
            , crop
        FROM farmers_fields 
        WHERE id='{field_id}'
    """
    reply = con.sql(query_fetch).df()
    if len(reply)!=1: 
        return {"status": "error", "error_message": "Field does not exist"}
  
    crop_val = reply.iloc[0]['crop']
    
    geohash_val = geohash.encode(reply.iloc[0]['latitude'], reply.iloc[0]['longitude'], precision=7)
    #print(f"({reply.iloc[0]['latitude']}, {reply.iloc[0]['longitude']}) => {geohash_val}")

    ## Find field measurements
    y_time = reply.iloc[0]['plant_ts']
    str_dtime = f"{y_time.year}-{y_time.month:02d}-{y_time.day:02d}T{y_time.hour:02d}:00:00"
    plant_ts = datetime.datetime.fromisoformat(str_dtime)
    
    ## Extract daily min and max temperature since plant_ts   
    query_fetch = f"""
    WITH raw_data as(
        SELECT id
            , ts
            , dt_year
            , dt_month
            , dt_day
            , dt_hour
            , temperature_2m
            , relative_humidity_2m
            , wind_speed_10m 
        FROM historical_measurements
        WHERE geohash='{geohash_val}'
            and ts >= '{plant_ts}'
    ) select dt_year, dt_month, dt_day 
        , min(temperature_2m) as min_temperature
        , max(temperature_2m) as max_temperature 
    from raw_data
    group by dt_year, dt_month, dt_day
    order by dt_year, dt_month, dt_day
    """
    reply = con.sql(query_fetch).df()
    t_base = 7
    if t_base_lookup.get(crop_val) != None:
        t_base = t_base_lookup.get(crop_val)
    # Get Accumulated GDD
    acc_gdd = get_gdd(reply['min_temperature'].tolist(),reply['max_temperature'].tolist(),t_base)
    response = {"status":"success", "number_of_samples": len(reply['min_temperature']), "acc_gdd": acc_gdd}
    return response



In [197]:
def get_soil_moisture(field_id: int) -> dict:
    """
    Args:
        field_id : Field identifier in farmers_fields table
        
    Returns:
        Dictionary with status and 
        Success: {"status": "success", }
          Error: {"status": "error", "error_message": "Field does not exist"}
    """
    response = {}
    return response

## Write Field Farming Measurements MCP Server code 


In [183]:
%%writefile mcp_server/farm_measurements_server.py
import logging
import os
import json
#import httpx
from mcp.server.fastmcp import FastMCP
import requests
from pydantic import BaseModel, Field
from typing import Union, List, TypeVar
import numpy as np
import duckdb
import geohash
from datetime import datetime, timedelta


my_mcp_name = "mcp-rt-farm-measurements"

mcp = FastMCP(my_mcp_name)

async def get_blitecast_sv24(relativehumidity_hours: List[float],avg_temp_hours: List[float])->int:
    """Calculates the Daily Severity Value (SV) based on Blitecast rules.
    
    Args:
        relativehumidity_hours: Hourly Relative Humity in the last 24 hours
        avg_temp_hours: Hourly Average Temperature (F) in the last 24 fours
    
    Returns:
        int: The assigned Severity Value (0, 1, 2, or 3).
    """
    if(len(relativehumidity_hours)<10):
        return 0
    isRHHigh = [False for i in range(len(relativehumidity_hours))]
    #print(isRHHigh)
    isRHHigh = [True if val>=90 else False for val in relativehumidity_hours ]
    #print(isRHHigh)
    count_high = isRHHigh.count(True)
    print(f"count_high = {count_high}")
    
    if 10 <= count_high <= 15:
        avg_temp = np.mean(avg_temp_hours)
        print(f"avg_temp = {avg_temp}")
        if 45 <= avg_temp <= 54:
            return 1
        elif 55 <= avg_temp <= 77:
            return 2
        else:
            return 0
    elif count_high >= 16:
        avg_temp = np.mean(avg_temp_hours)
        print(f"avg_temp = {avg_temp}")
        if 45 <= avg_temp <= 54:
            return 2
        elif 55 <= avg_temp <= 77:
            return 3
        else:
            return 0
    
    return 0

@mcp.tool()
async def get_blitecast_sv(field_id: int)-> dict:
    """Calculates Blite Cast Severity Value of the given field 

        Args:
            field_id : Field id in farmers_fields table
            
        Returns:
            int [0,1,2,3]
            Dictionary with status and Blite Cast Severity Value.
            Success: {"status": "success", "number_of_samples":127, "BliteCast_SV": 0}
              Error: {"status": "error", "error_message": "Field does not exist"}
    """
    ## Find field 
    con = duckdb.connect('ai-farmer-buddy.db',read_only=True)
    query_fetch = f"""
        SELECT id
            , latitude 
            , longitude
        FROM farmers_fields 
        WHERE id='{field_id}'
    """
    reply = con.sql(query_fetch).df()
    if len(reply)<1: 
        return {"staus":"error", "error_message":f"Field ({field_id}) does not exist"}
    ##     
    geohash_val = geohash.encode(reply.iloc[0]['latitude'], reply.iloc[0]['longitude'], precision=7)
    #print(f"({reply.iloc[0]['latitude']}, {reply.iloc[0]['longitude']}) => {geohash_val}")

    ## Find data of field
    yesterday_utc_time = datetime.today() - timedelta(days=1)
    y_time = yesterday_utc_time
    str_dtime=f"{y_time.year}-{y_time.month:02d}-{y_time.day:02d}T{y_time.hour:02d}:00:00"
    query_fetch = f"""
        SELECT id
            , ts
            , temperature_2m
            , relative_humidity_2m
            , wind_speed_10m 
        FROM historical_measurements
        WHERE geohash='{geohash_val}'
            and ts >= '{str_dtime}'
    """
    reply = con.sql(query_fetch).df()
    if len(reply)<1:
        return {"staus":"error", "error_message": f"Field ({field_id}) does not have enough data!"}
    val = get_blitecast_sv24(reply['relative_humidity_2m'].tolist(),reply['temperature_2m'].tolist())    
    return { "status": "success", "number_of_samples":len(reply), "BliteCast_SV": val }


##-----------MCP: Accumulated GDD

t_base_lookup = {"tomato":7, "potato":10}

##

async def get_gdd(t_min: List[float], t_max: List[float], t_base: float)-> float:
    """Calculate Growing Degree Days 

        Daily metric.
        
    Args:
        t_min (float): Daily minimum temperature (°C).
        t_max (float): Daily maximum temperature (°C).
        t_base (float): Base temperature for the organism (°C).

    Returns:
        float : Daily GDD
    """
    if t_base is None:
        t_base=7.0
    t_diff = [ (((mx - mn)/2)-t_base) for mx, mn in zip(t_max, t_min)]        
    return max(0.0, sum(t_diff))

@mcp.tool()
async def get_accumlated_gdd(field_id: int)-> dict:
    """Calculates Accumulated GDD for the given field 

    Args:
        field_id : Field identifier in farmers_fields table
        
    Returns:
        Dictionary with status and Accumulated GDD Value.
        Success: {"status": "success", "number_of_samples":127, "acc_gdd": 0}
          Error: {"status": "error", "error_message": "Field does not exist"}

    """
    ## Find field 
    con = duckdb.connect('ai-farmer-buddy.db',read_only=True)
    query_fetch = f"""
        SELECT id
            , latitude 
            , longitude
            , plant_ts
            , crop
        FROM farmers_fields 
        WHERE id='{field_id}'
    """
    reply = con.sql(query_fetch).df()
    if len(reply)!=1: 
        return {"status": "error", "error_message": "Field does not exist"}
  
    crop_val = reply.iloc[0]['crop']
    
    geohash_val = geohash.encode(reply.iloc[0]['latitude'], reply.iloc[0]['longitude'], precision=7)
    #print(f"({reply.iloc[0]['latitude']}, {reply.iloc[0]['longitude']}) => {geohash_val}")

    ## Find field measurements
    y_time = reply.iloc[0]['plant_ts']
    str_dtime = f"{y_time.year}-{y_time.month:02d}-{y_time.day:02d}T{y_time.hour:02d}:00:00"
    plant_ts = datetime.datetime.fromisoformat(str_dtime)
    
    ## Extract daily min and max temperature since plant_ts   
    query_fetch = f"""
    WITH raw_data as(
        SELECT id
            , ts
            , dt_year
            , dt_month
            , dt_day
            , dt_hour
            , temperature_2m
            , relative_humidity_2m
            , wind_speed_10m 
        FROM historical_measurements
        WHERE geohash='{geohash_val}'
            and ts >= '{plant_ts}'
    ) select dt_year, dt_month, dt_day 
        , min(temperature_2m) as min_temperature
        , max(temperature_2m) as max_temperature 
    from raw_data
    group by dt_year, dt_month, dt_day
    order by dt_year, dt_month, dt_day
    """
    reply = con.sql(query_fetch).df()
    t_base = 7
    if t_base_lookup.get(crop_val) != None:
        t_base = t_base_lookup.get(crop_val)
    # Get Accumulated GDD
    acc_gdd = get_gdd(reply['min_temperature'].tolist(),reply['max_temperature'].tolist(),t_base)
    response = {"status":"success", "number_of_samples": len(reply['min_temperature']), "acc_gdd": acc_gdd}
    return response

##----------- MCP 

async def shutdown_event():
    logging.info(f"Shutting down MCP Server {my_mcp_name}")
    pass

if __name__ == "__main__":
    mcp_server_log_dir = "./mcp_server"
    # Clean up any previous logs
    for log_file in ["./mcp_server/logger.log", "./mcp_server/web.log", "./mcp_server/tunnel.log"]:
        if os.path.exists(log_file):
            os.remove(log_file)
            print(f"🧹 Cleaned up {log_file}")

    # Configure logging with DEBUG log level.
    logging.basicConfig(
        filename="./mcp_server/logger.log",
        level=logging.DEBUG,
        format="%(filename)s:%(lineno)s %(levelname)s:%(message)s",
    )
    ## 
    logging.info(f"Starting MCP Server {my_mcp_name}")
    mcp.run(transport="stdio")

Writing mcp_server/farm_measurements_server.py


In [198]:
!ls -al mcp_server/*

-rw-r--r-- 1 root root 7316 Dec  1 04:26 mcp_server/farm_measurements_server.py


In [185]:
!cat mcp_server/farm_measurements_server.py

import logging
import os
import json
#import httpx
from mcp.server.fastmcp import FastMCP
import requests
from pydantic import BaseModel, Field
from typing import Union, List, TypeVar
import numpy as np
import duckdb
import geohash
from datetime import datetime, timedelta


my_mcp_name = "mcp-rt-farm-measurements"

mcp = FastMCP(my_mcp_name)

async def get_blitecast_sv24(relativehumidity_hours: List[float],avg_temp_hours: List[float])->int:
    """Calculates the Daily Severity Value (SV) based on Blitecast rules.
    
    Args:
        relativehumidity_hours: Hourly Relative Humity in the last 24 hours
        avg_temp_hours: Hourly Average Temperature (F) in the last 24 fours
    
    Returns:
        int: The assigned Severity Value (0, 1, 2, or 3).
    """
    if(len(relativehumidity_hours)<10):
        return 0
    isRHHigh = [False for i in range(len(relativehumidity_hours))]
    #print(isRHHigh)
    isRHHigh = [True if val>=90 else False for val in relativehumidity_hours ]
    #p

In [188]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# Create server parameters for stdio connection
mcp_rt_farm_measurements_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your farm_measurements_server.py file
    args=["./mcp_server/farm_measurements_server.py"],
)

print("✅ RT Farming Measurements MCP Server created")

✅ RT Farming Measurements MCP Server created


In [187]:
mcp_rt_farm_measurements_server_params

StdioServerParameters(command='python', args=['./mcp_server/farm_measurements_server.py'], env=None, cwd=None, encoding='utf-8', encoding_error_handler='strict')

In [189]:
import asyncio
from typing import Optional
from contextlib import AsyncExitStack

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

exit_stack = AsyncExitStack()
stdio_transport = await exit_stack.enter_async_context(stdio_client(mcp_rt_farm_measurements_server_params))
stdio, write = stdio_transport
session = await exit_stack.enter_async_context(ClientSession(stdio, write))
    
await session.initialize()
    
# List available tools
response = await session.list_tools()
tools = response.tools
print("\nConnected to server with tools:", [tool.name for tool in tools])

available_tools = [{ 
        "name": tool.name,
        "description": tool.description,
        "input_schema": tool.inputSchema
    } for tool in response.tools]

print(available_tools)



Connected to server with tools: ['get_blitecast_sv', 'get_accumlated_gdd']
[{'name': 'get_blitecast_sv', 'description': 'Calculates Blite Cast Severity Value of the given field \n\n        Args:\n            field_id : Field id in farmers_fields table\n            \n        Returns:\n            int [0,1,2,3]\n            Dictionary with status and Blite Cast Severity Value.\n            Success: {"status": "success", "number_of_samples":127, "BliteCast_SV": 0}\n              Error: {"status": "error", "error_message": "Field does not exist"}\n    ', 'input_schema': {'properties': {'field_id': {'title': 'Field Id', 'type': 'integer'}}, 'required': ['field_id'], 'title': 'get_blitecast_svArguments', 'type': 'object'}}, {'name': 'get_accumlated_gdd', 'description': 'Calculates Accumulated GDD for the given field \n\n    Args:\n        field_id : Field identifier in farmers_fields table\n        \n    Returns:\n        Dictionary with status and Accumulated GDD Value.\n        Success: {

'\ntool_name = \'get_current_temperature\'\ntool_args = { \'latitude\': 28, \'longitude\':28 }\nresult = await session.call_tool(tool_name, tool_args)\n\ntool_results = {"call": tool_name, "result": result}\nfinal_text = f"[Calling tool {tool_name} with args {tool_args}]"\nprint(final_text)\nprint(tool_results)\n'

In [190]:
tool_name = 'get_blitecast_sv'
tool_args = { 'field_id': 2 }
result = await session.call_tool(tool_name, tool_args)

tool_results = {"call": tool_name, "result": result}
final_text = f"[Calling tool {tool_name} with args {tool_args}]"
print(final_text)
print(tool_results)

[Calling tool get_blitecast_sv with args {'field_id': 2}]
{'call': 'get_blitecast_sv', 'result': CallToolResult(meta=None, content=[TextContent(type='text', text='Error executing tool get_blitecast_sv: IO Error: Could not set lock on file "/kaggle/working/ai-farmer-buddy.db": Conflicting lock is held in /usr/bin/python3.11 (PID 47). See also https://duckdb.org/docs/stable/connect/concurrency', annotations=None, meta=None)], structuredContent=None, isError=True)}


In [39]:
from google.adk.tools.mcp_tool import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

# MCP integration with Real-Time Farming Measurements Server
mcp_rt_farming_measurements_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="python",  # Run MCP server via python 
            args=[ "./mcp_server/farm_measurements_server.py",],
            #tool_filter=["getTinyImage"],
        ),
        timeout=30,
    )
)

print("✅ MCP Tool created for Real-Time Farming Measurements")

✅ MCP Tool created for Real-Time Farming Measurements


In [40]:
from google.adk.agents import LlmAgent

# Farming measurements agent with custom function tools
instruction_prompt="""
 You are an experienced farming advisor. 
 Use 'get_blitecast_sv()' to find Blite cast severity value of given field by using field_id   
 Use 'get_accumlated_gdd()' to find Accumulated GDD value of given field by using field_id
"""

rt_farming_measurements_agent = LlmAgent(
    name="rt_farming_measurements_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction=instruction_prompt,
    tools=[mcp_rt_farming_measurements_server, ],
)

In [199]:
import duckdb

def get_fields_of_farmer_by_name(farmer_name: str)-> dict:
    """Retrives farmer's fields by using farmer's name. 
    
    Information of each field contains crop kind, planting time, latitude, longitude, and field_id

    Args:
        farmer_name: farmer's name 
        
    Returns:
        Dictionay object is returned
        {"status":"success", "farmer_name": "bob", "farmer_id": 1, 
                "fields":[{"crop":"","latitude":,"longitude":,"plant_ts":, "field_id": }] }
        {"status":"error", "error_message":"Farmer does not exist!"}
    
    """
    response = {"status":"error", "error_message": f"Farmer {farmer_name} does not exist!"}
    if farmer_name is None:
        return response
    if len(farmer_name.strip()) < 1:
        return response
    try:
        # Get from database
        with duckdb.connect('ai-farmer-buddy.db') as con:
            query_fields=f"""
            SELECT f.id, f.name 
            FROM farmers f
            WHERE f.name='{farmer_name.strip().lower()}'
            """
            df = con.sql(query_fields).df()
            if len(df)>0:
                farmer_id = df.iloc[0]['id']
                farmer_nm = df.iloc[0]['name']
                response = {"status": "success", "farmer_name": farmer_nm, "farmer_id": farmer_id}
                query_fields=f"""
                        SELECT ff.id as field_id
                            , ff.farmer_id
                            , ff.crop
                            , ff.latitude
                            , ff.longitude
                            , ff.plant_ts
                        FROM farmers_fields ff
                        WHERE ff.farmer_id={farmer_id}
                    """
                farmer_fields=[]
                df = con.sql(query_fields).df()
                if len(df) > 0:
                    for i in range(len(df)):
                        dstr = {}
                        dstr['field_id']  = df.iloc[i]['field_id']
                        dstr['crop']      = df.iloc[i]['crop']
                        dstr['latitude']  = df.iloc[i]['latitude']
                        dstr['longitude'] = df.iloc[i]['longitude']
                        dstr['plant_ts']  = f"{df.iloc[i]['plant_ts']}"
                        farmer_fields.append(dstr)
                # 
                if len(farmer_fields) > 0 :
                    response['fields']=farmer_fields       
                else:
                    response = {"status":"error", "error_message": f"Farmer {farmer_name} has too many records!"}
    except Exception as ex:
        response['error_message']=f"Exception : {ex}"

    return response
    

In [200]:
ff_name='bob'
ans=get_fields_of_farmer_by_name(ff_name)
print(f"fields of farmer {ff_name}=>{ans}")

fields of farmer bob=>{'status': 'success', 'farmer_name': 'bob', 'farmer_id': 1, 'fields': [{'field_id': 1, 'crop': 'tomato', 'latitude': -29.65182, 'longitude': 27.088251, 'plant_ts': '2025-10-21 17:13:00'}, {'field_id': 2, 'crop': 'potato', 'latitude': -29.639688, 'longitude': 27.081528, 'plant_ts': '2025-10-22 17:53:00'}]}


## Translator Agent

Translate from English text to Sesotho.


In [211]:
from google.adk.agents import LlmAgent

# Translator agent 
instruction_translator_prompt="""
 You are an experienced translator. Translate given English text to Sesotho. 
"""

rt_farming_translator_agent = LlmAgent(
    name="rt_farming_translator_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction=instruction_translator_prompt,
    #tools=[mcp_rt_farming_measurements_server, ],
)


### Test Translator Agent (English text to Sesotho)

In [212]:
from google.genai import types as genai_types

user_id = "user_123"
session_id = "conversation_abc"

sentence = "Hello! How are you? What is your name?"
# Create the user's message
user_message = genai_types.Content(
    role="user", parts=[genai_types.Part(text=sentence)]
)

# Test the translator agent
translator_agent_runner = InMemoryRunner(agent=rt_farming_translator_agent)
_ = await translator_agent_runner.run_debug(sentence,user_id=user_id,session_id=session_id
)


 ### Created new session: conversation_abc

User > Hello! How are you? What is your name?
rt_farming_translator_agent > Dumela! O phela joang? Lebitso la hao ke mang?


In [213]:
translator_agent_runner.close()

<coroutine object Runner.close at 0x7ad8589d12a0>

# Farming Alerts Agent

Farming alerts agent evaluates the farming measurements with respect to the crop's needs to decide risk level. 

In [201]:
from google.adk.agents import LlmAgent

# Translator agent 
instruction_farming_alerts_prompt="""
 You are an experienced farming advisor. 
 Here are some examples to evaluate alert level based on crop, accumulated gdd.
 <examples>
     <example>
         <input crop='tomato' accumulated_gdd='150' field_id='2'/>
         <alert>High risk, accumulated gdd is high for tomato in field 2.</alert>
     </example>
     <example>
         <input crop='tomato' accumulated_gdd='100' field_id='2'/>
         <alert>Medium risk for tomato, monitor field 2 conditions closely for moisture.</alert>
     </example>
     <example>
         <input crop='tomato' accumulated_gdd='120' field_id='3'/>
         <alert>Medium risk for tomato, monitor field 3 conditions closely for moisture.</alert>
     </example>
     <example>
         <input crop='tomato' accumulated_gdd='90' field_id='2'/>
         <alert>Conditions in field 2 is good for tomato.</alert>
     </example>
     <example>
         <input crop='tomato' accumulated_gdd='80' field_id='3'/>
         <alert>Conditions in field 3 is good for tomato.</alert>
     </example>
     <example>
         <input crop='potato' accumulated_gdd='250' field_id='2'/>
         <alert>High risk, accumulated gdd is high for potato in field 2.</alert>
     </example>
     <example>
         <input crop='potato' accumulated_gdd='200' field_id='3'/>
         <alert>High risk, accumulated gdd is high for potato in field 3.</alert>
     </example>
     <example>
         <input crop='potato' accumulated_gdd='170' field_id='2'/>
         <alert>Medium risk, for potato, monitor field 2 conditions closely for moisture.</alert>
     </example>
     <example>
         <input crop='potato' accumulated_gdd='150' field_id='3'/>
         <alert>Medium risk, for potato, monitor field 3 conditions closely for moisture.</alert>
     </example>
     <example>
         <input crop='potato' accumulated_gdd='120' field_id='2'/>
         <alert>Conditions in field 2 is good for potato.</alert>
     </example>
 </examples>

 Evaluate and give appropriate alerting information for given field by using the crop and farming metrics.   
 
"""

rt_farming_alerts_agent = LlmAgent(
    name="rt_farming_alerts_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction=instruction_farming_alerts_prompt,
)


# Farming Recommendation Agent

Farming Recommendation Agent advises farmer based on field's farming measurements (past, current, forecasted) and crop on the specific field. 



In [33]:
from google.adk.agents import LlmAgent

# Translator agent 
instruction_farming_recommendations_prompt="""
 You are an experienced farming advisor. 
 Here are some example recommendations on crop, accumulated gdd.
 <examples>
     <example>
         <input crop='tomato' accumulated_gdd='150' field_id='2'/>
         <recommendation>High risk, accumalted gdd is high for field 2.</recommendation>
     </example>
     <example>
         <input crop='tomato' accumulated_gdd='100' field_id='2'/>
         <recommendation>Monitor field 2 conditions closely for moisture.</recommendation>
     </example>
     <example>
         <input crop='tomato' accumulated_gdd='90' field_id='2'/>
         <recommendation>Conditions in field 2 seem fine.</recommendation>
     </example>
     <example>
         <input crop='potato' accumulated_gdd='250' field_id='2'/>
         <recommendation>High risk, accumalted gdd is high for field 2.</recommendation>
     </example>
     <example>
         <input crop='potato' accumulated_gdd='170' field_id='2'/>
         <recommendation>Medium risk, monitor field 2 conditions closely for moisture.</recommendation>
     </example>
     <example>
         <input crop='potato' accumulated_gdd='120' field_id='2'/>
         <recommendation>Low risk for field 2.</recommendation>
     </example>
 </examples>

 Evaluate and give appropriate recommendations for given field 
 by using the crop and farming metrics.   
 
"""

rt_farming_recommendations_agent = LlmAgent(
    name="rt_farming_recommendations_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction=instruction_farming_alerts_prompt,
)

# AI Farming Buddy (Coordinator Agent)

In [207]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types
# 
ai_farming_buddy_instructions="""
You are a farming advisor. 
Your goal is to inform farming alerts and give farming recommendations for each field a farmer has. 
1. First, you must call 'get_current_date_and_time()' to find the what day is today, YYYY/MM/DD.
2. Next, greet the farmer with 'Today is YYYY/MM/DD. What is your name?'
3. Next, you must call 'get_fields_of_farmer_by_name()' by using farmer's name to find the fields of farmer, crop on each field, latitude and longitude of each field 
4. Next, you must call 'weather_agent' to get the local weather for the farmer's fields.
5. Next, you must call 'rt_farming_measurements_agent' to collect farming measurements (such as accumulated GDD, Bligt Cast Severity Value) of the farmer's fields 
6. Next, you must call 'rt_farming_alerts_agent' to get alerts for each field growing the crop.
7. Next, you must call to get recommendations for each field growing the crop.
8. Next, you must call 'rt_farming_translator_agent' to translate from English alerts and recommendations to Sesotho. 
9. Finally, presents alerts and recommendations in Sesotho as your response.

hint: use 'hitl_collect_input()' to collect user's input when needed. 

Guardrails
If the question is not about weather or farming, politely say you are only trained to discuss weather or farming.

"""

ai_farming_buddy_agent = Agent(
    name="AI_Farming_Buddy_Coordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction=ai_farming_buddy_instructions,
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[get_current_date_and_time
           , get_fields_of_farmer_by_name
           , hitl_collect_input
           , AgentTool(weather_agent)
           , AgentTool(rt_farming_measurements_agent)
           , AgentTool(rt_farming_alerts_agent)
           , AgentTool(rt_farming_recommendations_agent)
           , AgentTool(rt_farming_translator_agent)
          ],
)

print("✅ AI_Farming_Buddy_Coordinator created.")

✅ AI_Farming_Buddy_Coordinator created.


In [208]:
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
) 

farming_buddy_agent_runner = InMemoryRunner(
                                app_name="AI_Farming_Buddy",
                                agent=ai_farming_buddy_agent, 
                                plugins=[LoggingPlugin()],
                            )

print("✅ AI_Farming_Buddy_Runner created.")

✅ AI_Farming_Buddy_Runner created.


In [209]:
# Test farming_buddy
user_id = "bob"
session_id = "conversation_abcd"
sentence = "Hello! My name is bob. How is the weather today?"

_ = await farming_buddy_agent_runner.run_debug(sentence,user_id=user_id,session_id=session_id)


 ### Created new session: conversation_abcd

User > Hello! My name is bob. How is the weather today?
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-0e722ddc-9963-497f-825e-a36aaa616079
[logging_plugin]    Session ID: conversation_abcd
[logging_plugin]    User ID: bob
[logging_plugin]    App Name: AI_Farming_Buddy
[logging_plugin]    Root Agent: AI_Farming_Buddy_Coordinator
[logging_plugin]    User Content: text: 'Hello! My name is bob. How is the weather today?'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-0e722ddc-9963-497f-825e-a36aaa616079
[logging_plugin]    Starting Agent: AI_Farming_Buddy_Coordinator
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: AI_Farming_Buddy_Coordinator
[logging_plugin]    Invocation ID: e-0e722ddc-9963-497f-825e-a36aaa616079


NameError: name 'nextval' is not defined

## Fields historical farming measurements (long-term context)

Real Time Farming Measurements leverages this historical measurments with current measurements. 

In [24]:
!pip install duckdb
!pip install python-geohash

  Preparing metadata (setup.py) ... done
  Created wheel for python-geohash: filename=python_geohash-0.8.5-cp311-cp311-linux_x86_64.whl size=41945 sha256=09aa0f6110bd6096fdfd1d9fe476d9d2f865474573472d253d5e37c4f5c0e061
  Stored in directory: /root/.cache/pip/wheels/02/7a/f4/c27d535af1a4ad8c1112e1bf299d9d47b57fe0fa2a464e4795
Successfully built python-geohash


In [2]:
import duckdb

In [121]:
con = duckdb.connect('ai-farmer-buddy.db')

In [122]:
results = con.sql("SHOW ALL TABLES").fetchall()
print(type(results))
for result in results:
    print(result)

<class 'list'>


## Configure farmers table 

Solution will utilize name field to discern known farmers. 

Expecting that conversation will start by: 'Hi, I am bob,...."

In [123]:
query="""
DROP TABLE IF EXISTS farmers;
"""
reply = con.sql(query)
print(reply)

None


### Create farmers table

In [124]:
query="""
CREATE OR REPLACE TABLE farmers (
   id integer primary key,
   name varchar(16) NOT NULL
);
"""
reply = con.sql(query)
print(reply)

None


## Load data to farmers table



In [125]:
query_insert="""
INSERT INTO farmers (id, name)
VALUES ('1','bob'),
        ('2','joe'),
        ('3','nancy'),
        ('4','april');
"""
reply = con.sql(query_insert)
print(reply)

None


### Test data in farmers table

In [126]:
import pandas as pd

query_select="""
SELECT * from farmers;
"""
reply = con.sql(query_select).fetchall()
#print(reply)

df=pd.DataFrame(reply, columns=['farmer_id', 'farmer_name'])
print(df)

   farmer_id farmer_name
0          1         bob
1          2         joe
2          3       nancy
3          4       april


## Configure farmers_field table

Assumption is that each farmer could have multiple fields. 
- Each field has one type of crop (tomato or patato).
- Each field has location. 
- Each field has plant date representing when the crop is planted. This information is utilized to calculate metrics based on the past, current, and predicted weather and soil measurements for this location. 

In [127]:
query="""
DROP TABLE IF EXISTS farmers_fields;
"""
reply = con.sql(query)
print(reply)

None


### Create farmers_fields table

In [177]:
query="""
CREATE OR REPLACE TABLE farmers_fields (
  id int,
  farmer_id int NOT NULL,
  crop VARCHAR(32) NOT NULL,
  latitude float NOT NULL, 
  longitude float NOT NULL,
  plant_ts timestamp NOT NULL
);
"""
reply = con.sql(query)
print(reply)

None


### Load data to farmers_field table

In [178]:
query_insert="""
INSERT INTO farmers_fields (id, farmer_id, crop, latitude, longitude, plant_ts)
VALUES ('1',1, 'tomato', -29.651820, 27.088251, '2025-10-21T17:13:00'),
       ('2',1, 'potato', -29.63968921422942, 27.08152759870933, '2025-10-22T17:53:00'),
       ('3',2, 'tomato', -29.600105, 27.117532, '2025-10-19T13:33:00'),
       ('4',2, 'potato', -29.604708, 27.120709, '2025-10-21T16:24:00'),
       ('5',3, 'tomato', -29.576853, 27.202513, '2025-10-10T10:50:00'),
       ('6',3, 'potato', -29.575702, 27.207411, '2025-10-11T11:43:00'),
       ('7',4, 'tomato', -29.496025, 27.333193, '2025-11-01T13:13:00'),
       ('8',4, 'potato', -29.500457, 27.332829, '2025-11-01T15:27:00'),    
"""
reply = con.sql(query_insert)
print(reply)

None


### Test data in farmers_fields table

In [179]:
query_select="""
SELECT f.id as farmer_id
    , ff.id as field_id
    , f.name as farmer_name
    , ff.crop 
    , ff.latitude
    , ff.longitude
    , ff.plant_ts
from farmers_fields ff
Join farmers f on ff.farmer_id=f.id
order by f.name;
"""
reply = con.sql(query_select).df()
print(reply)

   farmer_id  field_id farmer_name    crop   latitude  longitude  \
0          4         7       april  tomato -29.496025  27.333193   
1          4         8       april  potato -29.500458  27.332829   
2          1         1         bob  tomato -29.651819  27.088251   
3          1         2         bob  potato -29.639688  27.081528   
4          2         3         joe  tomato -29.600105  27.117533   
5          2         4         joe  potato -29.604708  27.120708   
6          3         5       nancy  tomato -29.576853  27.202513   
7          3         6       nancy  potato -29.575703  27.207411   

             plant_ts  
0 2025-11-01 13:13:00  
1 2025-11-01 15:27:00  
2 2025-10-21 17:13:00  
3 2025-10-22 17:53:00  
4 2025-10-19 13:33:00  
5 2025-10-21 16:24:00  
6 2025-10-10 10:50:00  
7 2025-10-11 11:43:00  


## Configure historical measurements data 

Simple model of measurements (facts table in dimensional modeling). 

historical_measurements table has the following columns
- **geohash** of field's location is used (at precision 7)
- **time dimension** is unpacked for different aggregation granularity
- **temperature_2m**: temperature above the surface
- **relative_humidity_2m**: humidity above the surface
- **wind_speed_10m**: wind speed above the surface

Details of measurments data please refer to [Open Meteo's Forecast API](https://open-meteo.com/en/docs)

In [131]:
query="""
CREATE OR REPLACE TABLE historical_measurements (
  id INTEGER primary key,
  geohash varchar(16) NOT NULL,
  ts timestamp,
  dt_year int,
  dt_month int,
  dt_day int,
  dt_hour int, 
  temperature_2m float,
  relative_humidity_2m float,
  wind_speed_10m float
);
"""
reply = con.sql(query)
print(reply)

None


Sequence allows us to implement AUTOINCREMENT functionality with nextval() function in DuckDb.  

In [210]:
query="""
CREATE SEQUENCE historical_measurements_id START 1;
"""
reply = con.sql(query)
print(reply)

CatalogException: Catalog Error: Sequence with name "historical_measurements_id" already exists!

In [133]:
query_select="""
SELECT f.id as farmer_id
    , ff.id as field_id
    , f.name as farmer_name
    , ff.latitude
    , ff.longitude
from farmers_fields ff
Join farmers f on ff.farmer_id=f.id
order by f.name;
"""
reply = con.sql(query_select).df()
print(reply)

   farmer_id  field_id farmer_name   latitude  longitude
0          4         7       april -29.496025  27.333193
1          4         8       april -29.500458  27.332829
2          1         1         bob -29.651819  27.088251
3          1         2         bob -29.639688  27.081528
4          2         3         joe -29.600105  27.117533
5          2         4         joe -29.604708  27.120708
6          3         5       nancy -29.576853  27.202513
7          3         6       nancy -29.575703  27.207411


In [136]:
import geohash

df= reply
df['geohash'] = df.apply(lambda row:  geohash.encode(row['latitude'], row['longitude'], precision=7),axis=1)
df

,farmer_id,field_id,farmer_name,latitude,longitude,geohash
0,4,7,april,-29.496025,27.333193,kdg2jyr
1,4,8,april,-29.500458,27.332829,kdg2jvx
2,1,1,bob,-29.651819,27.088251,kder26j
3,1,2,bob,-29.639688,27.081528,kder2k2
4,2,3,joe,-29.600105,27.117533,kder956
5,2,4,joe,-29.604708,27.120708,kder94s
6,3,5,nancy,-29.576853,27.202513,kderep8
7,3,6,nancy,-29.575703,27.207411,kderepg


In [139]:
import geohash
import pandas as pd
from dateutil import parser

query_insert_start="""
INSERT INTO historical_measurements(id, geohash, ts, dt_year, dt_month, dt_day, dt_hour, 
  temperature_2m,
  relative_humidity_2m,
  wind_speed_10m)
VALUES   
"""
## For each field 
for i in range(len(df)):
    print(f"{df.iloc[i]['field_id']}-{df.iloc[i]['geohash']}")
    ## Get Last 10 days hourly data
    res=get_historical_weather_tool(df.iloc[i]['latitude'],df.iloc[i]['longitude'],10)
    if res['status']!='success':
        print(f" Error: {res}")
        continue
        
    ## Parse query result into dataframe
    df_res=pd.DataFrame( {'time': res['results']['hourly']['time'], 
                'temperature_2m': res['results']['hourly']['temperature_2m'],
                'relative_humidity_2m': res['results']['hourly']['relative_humidity_2m'],
                'wind_speed_10m': res['results']['hourly']['wind_speed_10m'],}
                )
    ## Geohash of field's location
    df_res['geohash'] = df_res.apply(lambda row:  geohash.encode(df.iloc[i]['latitude'],df.iloc[i]['longitude'], precision=7),axis=1)
    ## Build INSERT
    query_insert = query_insert_start
    for j in range(len(df_res)):
        dtime = parser.parse(df_res.iloc[i]['time'])
        str = f"(nextval('historical_measurements_id'),'{df_res.iloc[j]['geohash']}','{df_res.iloc[j]['time']}',{dtime.year},{dtime.month},{dtime.day},{dtime.hour},{df_res.iloc[j]['temperature_2m']},{df_res.iloc[j]['relative_humidity_2m']},{df_res.iloc[j]['wind_speed_10m']}),"
        query_insert = "".join([query_insert, str])
    if len(df_res)>0:
        reply = con.sql(query_insert)
        print(reply)

7-kdg2jyr
{'latitude': -29.5, 'longitude': 27.375, 'generationtime_ms': 2.1104812622070312, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 1520.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C', 'relative_humidity_2m': '%', 'wind_speed_10m': 'km/h'}, 'hourly': {'time': ['2025-11-21T00:00', '2025-11-21T01:00', '2025-11-21T02:00', '2025-11-21T03:00', '2025-11-21T04:00', '2025-11-21T05:00', '2025-11-21T06:00', '2025-11-21T07:00', '2025-11-21T08:00', '2025-11-21T09:00', '2025-11-21T10:00', '2025-11-21T11:00', '2025-11-21T12:00', '2025-11-21T13:00', '2025-11-21T14:00', '2025-11-21T15:00', '2025-11-21T16:00', '2025-11-21T17:00', '2025-11-21T18:00', '2025-11-21T19:00', '2025-11-21T20:00', '2025-11-21T21:00', '2025-11-21T22:00', '2025-11-21T23:00', '2025-11-22T00:00', '2025-11-22T01:00', '2025-11-22T02:00', '2025-11-22T03:00', '2025-11-22T04:00', '2025-11-22T05:00', '2025-11-22T06:00', '2025-11-22T07:00', '2025-11-22T08:00', '2025-11-22T09:

In [140]:
query_agg = """
SELECT geohash, count(*) as cnt
FROM historical_measurements
GROUP BY geohash
"""
reply = con.sql(query_agg).df()
reply

,geohash,cnt
0,kdg2jvx,408
1,kder26j,408
2,kderepg,408
3,kdg2jyr,408
4,kder2k2,408
5,kderep8,408
6,kder956,408
7,kder94s,408


# Appendix

Collected code details. 

## Setup

### MCP Server (weather)

In [35]:
!mkdir mcp_server

In [ ]:
!npm install @modelcontextprotocol/inspector@0.15.0

In [ ]:
!npx @modelcontextprotocol/inspector uv --directory mcp_server run weather_server.py

### Test get_historical_weather_tool()


In [ ]:
import pandas as pd

res = get_historical_weather_tool(28,28,1)

print(type(res))
print(res['status'])
#print(res['results'])  , 
#print(res['results']['hourly'])
df=pd.DataFrame( {'time': res['results']['hourly']['time'], 
                'temperature_2m': res['results']['hourly']['temperature_2m'],
                'relative_humidity_2m': res['results']['hourly']['relative_humidity_2m'],
                'wind_speed_10m': res['results']['hourly']['wind_speed_10m'],}
                )
df

### Blitecast Severity Value Tests

In [194]:
relativehumidity_hours_test = [90, 91, 50, 93, 94, 93, 92, 91, 91, 91, 90, 90, 90, 91, 50, 93, 94, 93, 92, 91, 91, 91, 90, 90 ]
avg_temp_hours_test = [45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45 ]
ans= get_blitecast_sv24(relativehumidity_hours_test,avg_temp_hours_test)
print(f"ans = {ans}")

relativehumidity_hours_test = [90, 91, 50, 93, 94, 93, 92, 91, 91, 91, 90, 90, 90, 91, 50, 93, 94, 93, 92, 91, 91, 91, 90, 90 ]
avg_temp_hours_test = [57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 60, 60, 60, 60, 60, 60, 60, 60, 60 ]
ans= get_blitecast_sv24(relativehumidity_hours_test,avg_temp_hours_test)
print(f"ans = {ans}")

relativehumidity_hours_test = [80, 81, 50, 83, 84, 83, 82, 81, 91, 91, 90, 90, 90, 91, 50, 93, 94, 93, 92, 91, 91, 91, 90, 90 ]
avg_temp_hours_test = [57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 60, 60, 60, 60, 60, 60, 60, 60, 60 ]
ans= get_blitecast_sv24(relativehumidity_hours_test,avg_temp_hours_test)
print(f"ans = {ans}")

relativehumidity_hours_test = [80, 81, 50, 83, 84, 83, 82, 81, 91, 91, 90, 90, 90, 91, 50, 93, 94, 93, 92, 91, 91, 91, 90, 90 ]
avg_temp_hours_test = [45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45 ]
ans= get_blitecast_sv24(relativehumidity_hours_test,avg_temp_hours_test)
print(f"ans = {ans}")

count_high = 22
avg_temp = 45.0
ans = 2
count_high = 22
avg_temp = 58.125
ans = 3
count_high = 15
avg_temp = 58.125
ans = 2
count_high = 15
avg_temp = 45.0
ans = 1


In [195]:
field_id = [1, 2, 3, 4, 5, 6, 7]
for v in field_id:
    print(f"field_id={v} => get_blitecast_sv({v})={get_blitecast_sv(v)}")


count_high = 13
avg_temp = 18.62978721679525
field_id=1 => get_blitecast_sv(1)={'status': 'success', 'number_of_samples': 188, 'BliteCast_SV': 0}
count_high = 13
avg_temp = 18.32978729491538
field_id=2 => get_blitecast_sv(2)={'status': 'success', 'number_of_samples': 188, 'BliteCast_SV': 0}
count_high = 13
avg_temp = 18.62978721679525
field_id=3 => get_blitecast_sv(3)={'status': 'success', 'number_of_samples': 188, 'BliteCast_SV': 0}
count_high = 13
avg_temp = 18.729787243173476
field_id=4 => get_blitecast_sv(4)={'status': 'success', 'number_of_samples': 188, 'BliteCast_SV': 0}
count_high = 20
avg_temp = 19.071808500492825
field_id=5 => get_blitecast_sv(5)={'status': 'success', 'number_of_samples': 188, 'BliteCast_SV': 0}
count_high = 20
avg_temp = 19.071808500492825
field_id=6 => get_blitecast_sv(6)={'status': 'success', 'number_of_samples': 188, 'BliteCast_SV': 0}
count_high = 26
avg_temp = 18.839361657487586
field_id=7 => get_blitecast_sv(7)={'status': 'success', 'number_of_samples'

In [ ]:
import duckdb
import datetime
import geohash

con = duckdb.connect('ai-farmer-buddy.db')

In [88]:
import geohash

field_id = 2
query_fetch = f"""
SELECT id
    , latitude 
    , longitude
    , plant_ts
FROM farmers_fields 
WHERE id='{field_id}'
"""
reply = con.sql(query_fetch).df()
print(reply)

geohash = geohash.encode(reply.iloc[0]['latitude'], reply.iloc[0]['longitude'], precision=7)
print(geohash)

   id   latitude  longitude            plant_ts
0   2 -29.639688  27.081528 2025-10-22 17:53:00
kder2k2


In [65]:
import datetime
from datetime import datetime, timedelta

field_id = 2

current_utc_time = datetime.utcnow()

yesterday_utc_time = datetime.today() - timedelta(days=1)

geohash = 'kder2k2'

print(f"{current_utc_time}-{yesterday_utc_time}")
y_time = yesterday_utc_time
str_dtime=f"{y_time.year}-{y_time.month:02d}-{y_time.day:02d}T{y_time.hour:02d}:00:00" 

dtime_24 = datetime.fromisoformat(str_dtime)
print(f"{current_utc_time}-{yesterday_utc_time}- [{str_dtime}] ==> {dtime_24}")

query_fetch = f"""
SELECT id
    , ts
    , temperature_2m
    , relative_humidity_2m
    , wind_speed_10m 
FROM historical_measurements
WHERE geohash='{geohash}'
    and ts >= '{str_dtime}'
"""

print(query_fetch)
reply = con.sql(query_fetch).df()
print(reply)
#reply = con.sql().fetchall()

2025-11-29 20:52:47.560118-2025-11-28 20:52:47.560162
2025-11-29 20:52:47.560118-2025-11-28 20:52:47.560162- [2025-11-28T20:00:00] ==> 2025-11-28 20:00:00

SELECT id
    , ts
    , temperature_2m
    , relative_humidity_2m
    , wind_speed_10m 
FROM historical_measurements
WHERE geohash='kder2k2'
    and ts >= '2025-11-28T20:00:00'

       id                  ts  temperature_2m  relative_humidity_2m  \
0    1461 2025-11-28 20:00:00       15.400000                  89.0   
1    1462 2025-11-28 21:00:00       14.500000                  83.0   
2    1463 2025-11-28 22:00:00       13.200000                  90.0   
3    1464 2025-11-28 23:00:00       13.300000                  84.0   
4    1465 2025-11-29 00:00:00       11.900000                  86.0   
..    ...                 ...             ...                   ...   
167  1628 2025-12-05 19:00:00       19.700001                  67.0   
168  1629 2025-12-05 20:00:00       19.400000                  68.0   
169  1630 2025-12-05 21:00

### Accumulated GDD Tests

In [98]:
import geohash

field_id = 2
query_fetch = f"""
SELECT id
    , latitude 
    , longitude
    , plant_ts
    , crop
FROM farmers_fields 
WHERE id='{field_id}'
"""
reply = con.sql(query_fetch).df()
print(reply)

geohash = geohash.encode(reply.iloc[0]['latitude'], reply.iloc[0]['longitude'], precision=7)
print(geohash)

   id   latitude  longitude            plant_ts    crop
0   2 -29.639688  27.081528 2025-10-22 17:53:00  patato
kder2k2


In [90]:
test_ts = reply.iloc[0]['plant_ts']

In [104]:
geohash = 'kder956'
# kder956, kder2k2
y_time = test_ts
str_dtime=f"{y_time.year}-{y_time.month:02d}-{y_time.day:02d}T{y_time.hour:02d}:00:00" 
plant_ts = datetime.datetime.fromisoformat(str_dtime)

query_fetch = f"""
with raw_data as(
SELECT id
    , ts
    , dt_year
    , dt_month
    , dt_day
    , dt_hour
    , temperature_2m
    , relative_humidity_2m
    , wind_speed_10m 
FROM historical_measurements
WHERE geohash='{geohash}'
    and ts >= '{plant_ts}'
) select dt_year, dt_month, dt_day 
        , min(temperature_2m) as min_temperature
        , max(temperature_2m) as max_temperature 
from raw_data
group by dt_year, dt_month, dt_day
order by dt_year, dt_month, dt_day
"""

print(query_fetch)
reply = con.sql(query_fetch).df()
print(reply)


with raw_data as(
SELECT id
    , ts
    , dt_year
    , dt_month
    , dt_day
    , dt_hour
    , temperature_2m
    , relative_humidity_2m
    , wind_speed_10m 
FROM historical_measurements
WHERE geohash='kder956'
    and ts >= '2025-10-22 17:00:00'
) select dt_year, dt_month, dt_day 
        , min(temperature_2m) as min_temperature
        , max(temperature_2m) as max_temperature 
from raw_data
group by dt_year, dt_month, dt_day
order by dt_year, dt_month, dt_day

   dt_year  dt_month  dt_day  min_temperature  max_temperature
0     2025        11      19              7.1             29.0


In [33]:
acc_gdd = get_accumlated_gdd(2)
acc_gdd

{'status': 'success', 'number_of_samples': 1, 'acc_gdd': 4.650000095367432}

### Database tests

In [ ]:
query="""
CREATE TABLE farmers (
  id int,
  name varchar(32) NOT NULL,
);
"""
reply = con.sql(query)
print(reply)

In [ ]:
query_insert="""
INSERT INTO historical_measurements(id, geohash, ts, dt_year, dt_month, dt_day, dt_hour, 
  temperature_2m,
  relative_humidity_2m,
  wind_speed_10m)
VALUES   
"""
geohash_7 = geohash.encode(-29.651820, 27.088251, precision=7)
geohash_7
for i in range(len(df)):
    dtime = parser.parse(timestamp)
    str = f"(nextval('historical_measurements_id'),'{geohash_7}','{df.iloc[0,0]}',{dtime.year},{dtime.month},{dtime.day},{dtime.hour},{df.iloc[0,1]},{df.iloc[0,2]},{df.iloc[0,3]}),"
    query_insert = "".join([query_insert, str])

print(query_insert)

## MISC

In [ ]:
from dateutil import parser

timestamp = df.iloc[0]['time']
print(f"timestamp = {timestamp}")

dtime = parser.parse(timestamp)
dtime

print(f"year({timestamp})  = {dtime.year}")
print(f"month({timestamp}) = {dtime.month}")
print(f"day({timestamp})   = {dtime.day}")
print(f"hour({timestamp})  = {dtime.hour}")

In [ ]:
import geohash

df  = reply
#geohash_code = geohash.encode(latitude, longitude, precision=12)
df['geohash_7'] = df.apply(lambda row:  geohash.encode(row['latitude'], row['longitude'], precision=7),axis=1)
df

In [ ]:
geohash_7 = geohash.encode(28, 28, precision=7)
geohash_7

In [44]:
chat_agent = Agent(
    name="ChattyAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a weather forecaster. 
    Your goal is to answer the user's questions about weather and weather forecast.
    1. First you must greet the user and ask user's name.
    2. Next, ask how you can help. ensure your question has user's name.
    3. Response user's questions about the weather and weather forecast politely.
        if the user's question is not about weather nor weather forecast, respond by 'I am only trained about weather or weather forecast!'
    """,
)

print("✅ ChattyAgent created.")

✅ ChattyAgent created.


In [111]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

chat_agent_runner = InMemoryRunner(agent=chat_agent)
#session_service = InMemorySessionService()
#chat_agent_runner = Runner(app_name="ChattyAgent",agent=chat_agent, session_service=session_service)

print("✅ Runner for ChattyAgent created.")

✅ Runner for ChattyAgent created.


In [112]:
response = await chat_agent_runner.run_debug(
     "Hi my name is Bob. What is the weather in NYC this week?"
)


 ### Created new session: debug_session_id

User > Hi my name is Bob. What is the weather in NYC this week?
ChattyAgent > Hello Bob! How can I help you today?

The weather in NYC this week is expected to be a mix of sun and clouds with temperatures generally in the mid-70s Fahrenheit. There's a chance of scattered showers on Wednesday and Thursday, so it might be a good idea to keep an umbrella handy.


In [113]:
response = await chat_agent_runner.run_debug(
    "I am traveling to NYC today. What should I wear for tomorrow?"
)


 ### Continue session: debug_session_id

User > I am traveling to NYC today. What should I wear for tomorrow?
ChattyAgent > Hello Bob! How can I help you today?

For tomorrow in NYC, I would recommend layers. Since the temperatures will be in the mid-70s, a light shirt or t-shirt would be comfortable. You might also want to bring a light jacket or sweater for the cooler parts of the day or if you plan to be out in the evening. Comfortable walking shoes are always a good idea for exploring the city!


In [114]:
response = await chat_agent_runner.run_debug(
    "I am traveling to NYC today. What should I eat tomorrow?"
)


 ### Continue session: debug_session_id

User > I am traveling to NYC today. What should I eat tomorrow?
ChattyAgent > I am only trained about weather or weather forecast!


In [115]:
from google.adk.sessions import InMemorySessionService, Session

async def generate_completion(user_prompt: str)->str:
    my_session = await session_service.create_session(app_name="ChattyAgent",user_id="bob")
    message = user_prompt ##"I am traveling to NYC today. What should I eat tomorrow?"
    user_content = types.Content(role='user', parts=[types.Part(text=message)])
    events = chat_agent_runner.run(user_id="bob",session_id=my_session.id,new_message=user_content)
    for event in events:
        print(f"\nDEBUG EVENT: {event}\n")
        if event.is_final_response() and event.content:
            response = event.content.parts[0].text.strip()

    return response[0].content.parts[0].text


prompt = "I am traveling to NYC today. What should I eat tomorrow?"
#answer = await generate_completion(prompt) 
answer = await chat_agent_runner.run_debug(prompt)
print(f"{answer}")


 ### Continue session: debug_session_id

User > I am traveling to NYC today. What should I eat tomorrow?
ChattyAgent > I am only trained about weather or weather forecast!
[Event(model_version='gemini-2.5-flash-lite', content=Content(
  parts=[
    Part(
      text='I am only trained about weather or weather forecast!'
    ),
  ],
  role='model'
), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=10,
  prompt_token_count=352,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=352
    ),
  ],
  total_token_count=362
), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e-2299d869-217

In [60]:
import gradio as gr
USER_ID='bob'
SESSION_ID='debug'

def chat_with_chatty(message, history):
    history = history or []
    messages = [{"role": h["role"], "content": h["content"]} for h in history]
    messages.append({"role": "user", "content": message})
    response='No LLM response'
    content = types.Content(role='user', parts=[types.Part(text=message)])
    events = chat_agent_runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)
    for event in events:
        print(f"\nDEBUG EVENT: {event}\n")
        if event.is_final_response() and event.content:
            response = event.content.parts[0].text.strip()
    #response =  chat_agent_runner.run_debug(message)
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": response})
    
with gr.Blocks() as ui_farmer_buddy:
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox(placeholder='Ask me anything...')
    msg.submit(chat_with_chatty, [msg,chatbot],[msg,chatbot])
    
ui_farmer_buddy.launch(share=True)

* Running on local URL:  http://127.0.0.1:7867
* Running on public URL: https://3b31cbdc88f11d903e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [96]:
import asyncio
import panel as pn
from panel.chat import ChatInterface
pn.extension("perspective")  # Enables Panel in Jupyter

In [119]:
import asyncio
import gradio as gr

def chat_with_chatty2(message):
    response='No LLM response'
    content = types.Content(role='user', parts=[types.Part(text=message)])
    events = chat_agent_runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)
    print(f"\nDEBUG EVENT: {events}\n")
    for event in events:
        print(f"\nDEBUG EVENT: {event}\n")
        if event.is_final_response() and event.content:
            response = event.content.parts[0].text.strip()
    
    return response


async def chat_response(message):
    #answer = await chat_agent_runner.run_debug(message)
    return f"You: {message}"
 
#iface = gr.Interface(fn=chat_with_chatty2, inputs="text", outputs="text")
iface = gr.Interface(fn=chat_response, inputs="text", outputs="text")
iface.launch()

* Running on local URL:  http://127.0.0.1:7876
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://f7202c77fc3627750e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [110]:
# chat_interface.py
import panel as pn

pn.extension()

async def callback(contents: str, user: str, instance: pn.chat.ChatInterface):
    #response = bot(contents)
    """
    print(f"sending {contents}")
    debug_str = f"sending {contents}"
    instance.send(debug_str,respond=False,)
    response = await chat_agent_runner.run_debug(contents)
    #yield response.content
    yield response
    """
    instance.send(contents,user=user,respond=False,)
    yield contents

chat_interface = pn.chat.ChatInterface(
    callback=callback, callback_user="user", 
    show_clear=False, show_undo=False, show_rerun=False
)
chat_interface.send(
    "Send a message to get a reply from the bot!",
    user="System",
    respond=False,
)
chat_interface.servable()

ChatInterface(_button_data={'send': _ChatButtonData(i...}, _buttons={'send': Button(align='cen...}, _input_container=Row, _input_layout=Row, _placeholder=ChatMessage, _widgets={'ChatAreaInput': ChatArea...}, callback=<function callback a..., callback_user='user', show_button_name=True, show_clear=False, show_rerun=False, show_undo=False, sizing_mode='stretch_width', widgets=[ChatAreaInput(css_classes...])
    [0] ChatMessage(object='Send a message to get a reply from the bot!', user='System', reactions=[])